In [ ]:
!pip install Scrapy --upgrade

In [ ]:
!scrapy startproject news

In [ ]:
!ls

In [ ]:
%cd /content/
!ls news/news

In [ ]:
%%writefile /content/news/news/items.py
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class NewsItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    headline = scrapy.Field()
    url = scrapy.Field()
    # pass


In [ ]:
%%writefile /content/news/news/pipelines.py
# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
# from itemadapter import ItemAdapter


# class NewsPipeline:
    # def process_item(self, item, spider):
        # return item

from scrapy import signals
from scrapy.exporters import CsvItemExporter

class NewsPipeline(object):

  def __init__(self):
    self.files = {}

  @classmethod
  def from_crawler(cls, crawler):
    pipeline = cls()
    crawler.signals.connect(pipeline.spider_opened, signals.spider_opened)
    crawler.signals.connect(pipeline.spider_closed, signals.spider_closed)
    return pipeline

  def spider_opened(self, spider):
    file = open('%s_items.csv' % spider.name, 'w+b')
    self.files[spider] = file
    self.exporter = CsvItemExporter(file)
    self.exporter.fields_to_export = ['headline', 'url']
    self.exporter.start_exporting()

  def spider_closed(self, spider):
    self.exporter.finish_exporting()
    file = self.files.pop(spider)
    file.close()

  def process_item(self, item, spider):
    self.exporter.export_item(item)
    return item

In [ ]:
%%writefile /content/news/news/settings.py
# Scrapy settings for news project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'news'

SPIDER_MODULES = ['news.spiders']
NEWSPIDER_MODULE = 'news.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'news (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = True

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'news.middlewares.NewsSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'news.middlewares.NewsDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   'news.pipelines.NewsPipeline': 300,
}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'


In [ ]:
%cd /content/
!ls news/news/spiders

In [1]:
%%writefile news/news/spiders/news_spider.py
import os
import csv

import scrapy
from bs4 import BeautifulSoup
from news.items import NewsItem

def get_urls_from_csv():
    with open('websites.csv', 'rbU') as csv_file:
        data = csv.reader(csv_file)
        scrapurls = []
        for row in data:
            scrapurls.append(row)
        return scrapurls

class NewsSpider(scrapy.Spider):
    name = "news"
    
    start_urls = [
                  "https://economictimes.indiatimes.com/topic/",
                  "https://timesofindia.indiatimes.com/topic/",
                  "https://www.ndtv.com/search?searchtext="
    ]
    
    start_urls = [url+keyword for url in start_urls for keyword in ['black money',
                                                                    'money laundering', 
                                                                    'money launder', 
                                                                    'lauder the money', 
                                                                    'money-mule', 
                                                                    'money mule', 
                                                                    'Hawala', 
                                                                    'drug-trafficking', 
                                                                    'drug trafficking', 
                                                                    'terror', 
                                                                    'terror financing'
    ]]

    def parse(self, response):
      # text data of entire webpage
      soup = BeautifulSoup(response.text, 'html.parser') # parse
      # extract body from it
      soup = soup.body # scrape.parse_soup

      all_links = soup.findAll("a", href=True)

      for link in all_links:
        news = NewsItem()
        if link.string and (response.url.split('/')[2] in link['href']):
          news['headline'] = link.string
          news['url'] = response.urljoin(link['href'])

          yield news

      # all_links = soup.findAll("a", {'target': '_blank'})

      # for link in all_links:
      #   news = NewsItem()
      #   news['headline'] = link.find('h2')
      #   if news['headline']:
      #     news['headline'] = news['headline'].string
      #     news['url'] = response.urljoin(link['href'])

      #     yield news

Writing news/news/spiders/news_spider.py


FileNotFoundError: ignored

In [ ]:
%cd /content/news/
!ls

In [ ]:
%%writefile websites.csv
https://economictimes.indiatimes.com/topic/,
https://timesofindia.indiatimes.com/topic/,
https://www.ndtv.com/search?searchtext=

In [ ]:
!scrapy crawl news

In [ ]:
!ls

In [ ]:
%%writefile economictimes/economictimes/spiders/articles.py
import scrapy
import os
import csv

from bs4 import BeautifulSoup
# from unidecode import unidecode
# import spacy

# nlp_Name = spacy.load("en_core_web_trf") # spacy.load(OUTPUT1)

# csv_columns = ['text', 'name']

# defining new variable passing two parameters
# writer = csv.DictWriter(open(os.path.abspath(os.path.join(os.getcwd(), 'test_results1.csv')), 'w'), fieldnames=csv_columns)

# writeheader() method to the write to the file object
# writer.writeheader()

class IndiaTestSpider(scrapy.Spider):
    name = "articles"

    def start_requests(self):
        urls = [
            'https://economictimes.indiatimes.com/topic/hdfc-bank-news',
            # 'http://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):

      articles = {}
      # text data of entire webpage
      soup = BeautifulSoup(response.text, 'html.parser') # parse
      # extract body from it
      soup = soup.body # scrape.parse_soup

      articles = soup.findAll("div", {"data-brcount": "24"})

      details = [paragraph for paragraph in articles.stripped_strings]

      details = ' '.join(details)

      articles['text'] = details

      yield articles

      # news_dict = {}

      # links = []

      # all_links = soup.findAll("a", {'target': '_blank'})

      # for link in all_links:
      #   headline = link.find('h2')
      #   if headline:
      #     news_dict['headline'] = headline.string
      #     news_dict['url'] = response.urljoin(link['href'])
      #     links.append(response.urljoin(link['href']))

      #     yield news_dict
      
      # for _link in links:
      #   yield _link

      # if len(soup.findAll('td'))==0:
      #   blocks = soup.findAll('div')
      # if len(soup.findAll('th'))==0: # this is the format
      #   blocks = soup.findAll('td') # of council of ministers webpage structure
      # else:
      #   blocks = soup.findAll('tr')

      # if len(soup.findAll('td'))==0:
      #   blocks = soup.findAll('div')
      # elif len(soup.findAll('th'))==0: # this is the format
      #   blocks = soup.findAll('td') # of council of ministers webpage structure
      # else:
      #   blocks = soup.findAll('tr')
      
      # blocks = list(set(blocks))
      
      # for block in blocks:
        # save into a dictionary
        # profile = {'text': '', 'name': ''}
        # content = block.get_text(separator=' ', strip=True).replace('\n', ' ').replace('\t', ' ')
        # content = unidecode(content)
        # print(content)
        # profile['text'] += content + ' '
        
        # name=''
        # doc = nlp_Name(content)
        # profile['text'] = content
        # for count,ent in enumerate(doc.ents):
          # if ent.label_ == 'PERSON':
            # profile['name'] +=ent.text+' '
            # if count==0:
              # break
        
        # writer.writerow(profile)

In [ ]:
%cd economictimes/
!scrapy crawl news -O news.json

In [ ]:
import pandas as pd